In [35]:
import torch
import torch.nn as nn
#from torch.nn.utils.rnn import pack_padded_sequence

class GatedRecurrentUnitHnNet(nn.Module):
    def __init__(self, input_size, output_size, 
                        window_size, hidden_size, num_layers, 
                        target_type_string='Regression',
                        bias=True, batch_first=True, 
                        bidirectional=False,
                        dropout_hidden=0, dropout_Hn=0):

        super().__init__()
        self.gru = nn.GRU(input_size=input_size,
                            hidden_size=hidden_size,
                            num_layers=num_layers,
                            bias=bias,
                            batch_first=batch_first,
                            dropout=dropout_hidden, 
                            bidirectional=bidirectional)
        self.hn_dropout_layer = nn.Dropout(dropout_Hn)
        self.predict_layer = nn.Linear(hidden_size * num_layers * (bidirectional + 1), output_size) #(hidden*layers*1, 1)
        
        self.target_type_string = target_type_string
        if target_type_string=='Regression':
            self.loss_function = nn.MSELoss()
        elif target_type_string=='Classification':
            self.loss_function = nn.CrossEntropyLoss()
             
    def forward(self, input):
        #print('batch:', input)
        #print(input.shape)
        #print(input[0])
        #print(input[0][0])
        #print(input[0][0][0])
        #print('input:', input)

        gru_out, h_n = self.gru(input) #shape: (num_layers * num_directions, batch, hidden_size)
        print('hn:', h_n)
        print('gru_out', gru_out)
        print(gru_out==h_n)
        #print('hn size:', h_n.size(0), h_n.size(1), h_n.size(2))
        
        list_to_cat = []
        for layers in h_n[:]:
            list_to_cat.append(layers)
        h_n_concat = torch.cat(list_to_cat, dim=1) #shape: (batch, hidden_size * num_layers * num_directions)
        print('hn concat:', h_n_concat)
        #print('hn concat size:', h_n_concat.size(0), h_n_concat.size(1))
        
        h_n_with_dropout = self.hn_dropout_layer(h_n_concat)
        predict = self.predict_layer(h_n_with_dropout) #predict est vertical
        #print('predict:', predict)
        return predict

In [36]:
input_size = 2
output_size = 1
window_size = 1
hidden_size = 8
num_layers = 1
target_type_string='Regression'
bias=True
batch_first=True
bidirectional=False
dropout_hidden=0
dropout_Hn=0

In [37]:
gru = GatedRecurrentUnitHnNet(input_size, output_size, 
                        window_size, hidden_size, num_layers, 
                        target_type_string='Regression',
                        bias=True, batch_first=True, 
                        bidirectional=False,
                        dropout_hidden=0, dropout_Hn=0)

In [38]:
inputs = [[[7.9018e+04, 1.2779e+01]],
        [[5.2380e+04, 1.9846e+01]],
        [[5.9300e+04, 1.0390e+01]],
        [[6.4618e+04, 7.1564e+00]],
        [[6.2786e+04, 6.4031e+00]],
        [[6.0496e+04, 6.3149e+00]],
        [[6.6420e+04, 1.0902e+01]],
        [[8.4788e+04, 1.6613e+01]],
        [[6.5484e+04, 9.7487e+00]],
        [[5.1308e+04, 2.7077e+01]],
        [[3.3682e+04, 1.7220e+01]]]

In [39]:
gru(torch.Tensor(inputs))

hn: tensor([[[ 1.,  1.,  0.,  1., -1.,  1.,  0.,  1.],
         [ 1.,  1.,  0.,  1., -1.,  1.,  0.,  1.],
         [ 1.,  1.,  0.,  1., -1.,  1.,  0.,  1.],
         [ 1.,  1.,  0.,  1., -1.,  1.,  0.,  1.],
         [ 1.,  1.,  0.,  1., -1.,  1.,  0.,  1.],
         [ 1.,  1.,  0.,  1., -1.,  1.,  0.,  1.],
         [ 1.,  1.,  0.,  1., -1.,  1.,  0.,  1.],
         [ 1.,  1.,  0.,  1., -1.,  1.,  0.,  1.],
         [ 1.,  1.,  0.,  1., -1.,  1.,  0.,  1.],
         [ 1.,  1.,  0.,  1., -1.,  1.,  0.,  1.],
         [ 1.,  1.,  0.,  1., -1.,  1.,  0.,  1.]]], grad_fn=<StackBackward>)
gru_out tensor([[[ 1.,  1.,  0.,  1., -1.,  1.,  0.,  1.]],

        [[ 1.,  1.,  0.,  1., -1.,  1.,  0.,  1.]],

        [[ 1.,  1.,  0.,  1., -1.,  1.,  0.,  1.]],

        [[ 1.,  1.,  0.,  1., -1.,  1.,  0.,  1.]],

        [[ 1.,  1.,  0.,  1., -1.,  1.,  0.,  1.]],

        [[ 1.,  1.,  0.,  1., -1.,  1.,  0.,  1.]],

        [[ 1.,  1.,  0.,  1., -1.,  1.,  0.,  1.]],

        [[ 1.,  1.,  0.,  1.,

tensor([[0.8084],
        [0.8084],
        [0.8084],
        [0.8084],
        [0.8084],
        [0.8084],
        [0.8084],
        [0.8084],
        [0.8084],
        [0.8084],
        [0.8084]], grad_fn=<AddmmBackward>)